# Milestone 2: 終極密碼遊戲 - 需求規格書

本文件詳細說明終極密碼遊戲的功能需求、技術規格與測試案例。

---

## 📋 Part I: 基本需求

### 需求 1.1：隨機數生成

**描述**：遊戲開始時，系統必須隨機產生一個整數作為答案。

**規格**：
- 使用 Python `random` 模組
- 預設範圍：1-100（包含 1 和 100）
- 每次遊戲開始時重新生成
- 不可在遊戲過程中讓玩家看到答案

**技術實作**：

In [ ]:
import random

# 正確做法
secret_number = random.randint(1, 100)  # 包含 1 和 100
print(f"隨機數已生成（測試用）: {secret_number}")

# ❌ 錯誤做法 1：範圍不正確
# secret_number = random.randint(1, 99)  # 無法產生 100

# ❌ 錯誤做法 2：使用 random() 未轉整數
# secret_number = random.random() * 100  # 會產生小數

**驗收標準**：
- [ ] 產生的數字在指定範圍內
- [ ] 包含邊界值（1 和 100）
- [ ] 每次執行產生不同數字（隨機性）

---

### 需求 1.2：玩家輸入處理

**描述**：系統必須能正確取得並驗證玩家輸入的猜測。

**規格**：
- 提示玩家輸入數字
- 顯示當前猜測次數
- 接受整數輸入
- 拒絕非數字輸入（顯示錯誤訊息，不計入次數）
- 拒絕範圍外數字（顯示錯誤訊息，不計入次數）

**技術實作**：

In [ ]:
# 正確做法：完整的輸入驗證
def get_valid_guess(min_num, max_num, attempt_num):
    """
    取得有效的玩家猜測
    
    參數：
        min_num: 最小值
        max_num: 最大值
        attempt_num: 當前猜測次數
    
    回傳：
        int: 有效的猜測數字
    """
    while True:
        try:
            guess = int(input(f"第 {attempt_num} 次猜測 ({min_num}-{max_num}): "))
            
            # 檢查範圍
            if guess < min_num or guess > max_num:
                print(f"❌ 請輸入 {min_num}-{max_num} 之間的數字！")
                continue
            
            return guess
            
        except ValueError:
            print("❌ 請輸入有效的數字！")

# 測試範例
# guess = get_valid_guess(1, 100, 1)
# print(f"有效猜測：{guess}")

**輸入驗證測試案例**：

In [ ]:
# 測試案例 1：正常數字
# 輸入：50
# 預期：接受，回傳 50

# 測試案例 2：邊界值
# 輸入：1 或 100
# 預期：接受，回傳該值

# 測試案例 3：範圍外數字
# 輸入：0 或 101
# 預期：顯示錯誤訊息，重新要求輸入

# 測試案例 4：非數字
# 輸入：abc 或 五十
# 預期：顯示錯誤訊息，重新要求輸入

# 測試案例 5：小數
# 輸入：50.5
# 預期：轉換為整數 50（或拒絕）

**驗收標準**：
- [ ] 正常數字能正確接受
- [ ] 邊界值（1, 100）能正確接受
- [ ] 範圍外數字會顯示錯誤，不計入次數
- [ ] 非數字輸入會顯示錯誤，不計入次數
- [ ] 錯誤訊息清楚友善

---

### 需求 1.3：猜測判斷邏輯

**描述**：系統必須比較玩家猜測與答案，並給予適當回饋。

**規格**：
- 猜測 > 答案：顯示「太大了！」
- 猜測 < 答案：顯示「太小了！」
- 猜測 = 答案：顯示「答對了！」並結束遊戲

**技術實作**：

In [ ]:
def check_guess(guess, secret):
    """
    檢查猜測結果
    
    參數：
        guess: 玩家猜測
        secret: 正確答案
    
    回傳：
        str: 'correct', 'too_high', 'too_low'
    """
    if guess == secret:
        return 'correct'
    elif guess > secret:
        return 'too_high'
    else:
        return 'too_low'

# 測試範例
secret = 63
print(check_guess(50, secret))  # 輸出：too_low
print(check_guess(75, secret))  # 輸出：too_high
print(check_guess(63, secret))  # 輸出：correct

**判斷邏輯測試案例**：

In [ ]:
# 測試案例組合
test_cases = [
    (50, 63, 'too_low', '太小了！'),
    (75, 63, 'too_high', '太大了！'),
    (63, 63, 'correct', '答對了！'),
    (1, 1, 'correct', '邊界測試：最小值'),
    (100, 100, 'correct', '邊界測試：最大值'),
]

for guess, secret, expected, description in test_cases:
    result = check_guess(guess, secret)
    status = "✅" if result == expected else "❌"
    print(f"{status} {description}: guess={guess}, secret={secret}, result={result}")

**驗收標準**：
- [ ] 三種判斷結果都正確
- [ ] 回饋訊息清楚易懂
- [ ] 邊界值判斷正確

---

### 需求 1.4：次數限制與計數

**描述**：遊戲必須限制猜測次數，並正確追蹤已使用次數。

**規格**：
- 預設最大次數：7 次
- 每次有效猜測計數器 +1
- 顯示當前是第幾次猜測
- 用完次數後結束遊戲
- 錯誤輸入不計入次數

**技術實作**：

In [ ]:
# 正確做法
MAX_ATTEMPTS = 7
attempts = 0

while attempts < MAX_ATTEMPTS:
    attempts += 1  # 先增加次數
    print(f"\n第 {attempts} 次猜測（剩餘 {MAX_ATTEMPTS - attempts} 次機會）")
    
    # 取得猜測...
    # 判斷結果...
    
    # 如果答對，跳出迴圈
    # if guess == secret:
    #     break

# 遊戲結束判斷
# if attempts >= MAX_ATTEMPTS:
#     print(f"\n很遺憾，你已用完 {MAX_ATTEMPTS} 次機會！")

**計數器測試案例**：

In [ ]:
# 模擬遊戲流程測試
def simulate_game(guesses, secret, max_attempts=7):
    """
    模擬遊戲測試計數器
    
    參數：
        guesses: 玩家猜測序列
        secret: 正確答案
        max_attempts: 最大次數
    """
    attempts = 0
    
    for guess in guesses:
        if attempts >= max_attempts:
            print(f"❌ 超過次數限制！")
            break
            
        attempts += 1
        print(f"第 {attempts} 次：猜測 {guess}", end=" ")
        
        if guess == secret:
            print(f"→ 答對了！總共 {attempts} 次")
            break
        elif guess > secret:
            print("→ 太大了！")
        else:
            print("→ 太小了！")
    
    if attempts >= max_attempts and guess != secret:
        print(f"\n遊戲結束！正確答案是 {secret}")

# 測試案例 1：3 次內猜中
print("=== 測試 1：3 次猜中 ===")
simulate_game([50, 75, 63], secret=63)

# 測試案例 2：用完 7 次
print("\n=== 測試 2：用完 7 次 ===")
simulate_game([10, 20, 30, 40, 50, 60, 70], secret=65)

# 測試案例 3：第 7 次猜中
print("\n=== 測試 3：第 7 次猜中 ===")
simulate_game([10, 20, 30, 40, 50, 60, 63], secret=63)

**驗收標準**：
- [ ] 計數器從 1 開始
- [ ] 每次猜測正確遞增
- [ ] 最多只能猜 7 次
- [ ] 答對後即停止，不繼續計數
- [ ] 顯示剩餘次數

---

### 需求 1.5：遊戲結束訊息

**描述**：遊戲結束時必須顯示適當的結果訊息。

**規格**：

**勝利訊息**（答對時）：
- 恭喜答對
- 顯示總共猜了幾次
- 給予鼓勵（可選）

**失敗訊息**（用完次數）：
- 告知遊戲結束
- 公布正確答案
- 鼓勵再試（可選）

**技術實作**：

In [ ]:
def show_victory_message(attempts, max_attempts):
    """顯示勝利訊息"""
    print("\n" + "="*50)
    print("🎉 恭喜答對了！")
    print(f"你總共猜了 {attempts} 次。")
    
    # 根據次數給予評價
    if attempts <= 3:
        print("💡 太厲害了！你是猜數字大師！")
    elif attempts <= 5:
        print("👍 表現不錯！")
    else:
        print("✅ 成功完成！")
    print("="*50)

def show_defeat_message(secret, max_attempts):
    """顯示失敗訊息"""
    print("\n" + "="*50)
    print(f"😢 很遺憾，你已用完 {max_attempts} 次機會！")
    print(f"正確答案是：{secret}")
    print("💪 別氣餒，再試一次吧！")
    print("="*50)

# 測試訊息
print("勝利訊息範例：")
show_victory_message(3, 7)

print("\n失敗訊息範例：")
show_defeat_message(63, 7)

**驗收標準**：
- [ ] 勝利訊息包含次數資訊
- [ ] 失敗訊息公布答案
- [ ] 訊息格式美觀易讀
- [ ] 包含適當的鼓勵文字

---

## 🚀 Part II: 進階需求

### 需求 2.1：難度選擇

**描述**：提供多種難度供玩家選擇。

**規格**：

| 難度 | 範圍 | 最大次數 |
|:-----|:-----|:---------|
| 簡單 | 1-50 | 10 次 |
| 中等 | 1-100 | 7 次 |
| 困難 | 1-200 | 6 次 |

**技術實作**：

In [ ]:
def choose_difficulty():
    """
    選擇遊戲難度
    
    回傳：
        tuple: (min_num, max_num, max_attempts)
    """
    print("\n=== 選擇難度 ===")
    print("1. 簡單 (1-50, 10 次機會)")
    print("2. 中等 (1-100, 7 次機會)")
    print("3. 困難 (1-200, 6 次機會)")
    
    while True:
        try:
            choice = int(input("\n請選擇 (1-3): "))
            
            if choice == 1:
                return (1, 50, 10)
            elif choice == 2:
                return (1, 100, 7)
            elif choice == 3:
                return (1, 200, 6)
            else:
                print("請輸入 1-3 之間的數字！")
        except ValueError:
            print("請輸入有效的數字！")

# 測試
# min_num, max_num, max_attempts = choose_difficulty()
# print(f"\n難度設定：範圍 {min_num}-{max_num}，{max_attempts} 次機會")

**驗收標準**：
- [ ] 提供至少 3 種難度選項
- [ ] 難度資訊顯示清楚
- [ ] 輸入驗證完整
- [ ] 正確回傳難度參數

---

### 需求 2.2：猜測歷史記錄

**描述**：記錄並顯示所有猜測歷史。

**規格**：
- 使用串列儲存每次猜測
- 每次猜測後顯示完整歷史
- 以易讀格式呈現

**技術實作**：

In [ ]:
def show_guess_history(history):
    """
    顯示猜測歷史
    
    參數：
        history: 猜測歷史串列
    """
    if not history:
        return
    
    print(f"\n猜測歷史：{history}")
    print(f"目前範圍：{min(history)} - {max(history)}")

# 測試範例
guess_history = []

# 模擬遊戲過程
guesses = [50, 75, 63]
for guess in guesses:
    guess_history.append(guess)
    print(f"猜測：{guess}")
    show_guess_history(guess_history)

**進階視覺化**：

In [ ]:
def show_visual_history(history, min_num, max_num):
    """
    視覺化猜測歷史（進階）
    
    顯示數字範圍與猜測位置
    """
    if not history:
        return
    
    print("\n猜測歷史視覺化：")
    for i, guess in enumerate(history, 1):
        # 計算位置百分比
        position = int((guess - min_num) / (max_num - min_num) * 40)
        bar = " " * position + "🔴"
        print(f"{i}. {guess:>3} |{bar}")
    
    print(f"      {min_num}{'':>38}{max_num}")

# 測試
test_history = [50, 75, 60, 68, 63]
show_visual_history(test_history, 1, 100)

**驗收標準**：
- [ ] 正確記錄所有猜測
- [ ] 顯示格式清晰
- [ ] 提供有用的統計資訊
- [ ] 視覺化呈現（加分項）

---

### 需求 2.3：重新遊戲功能

**描述**：遊戲結束後詢問是否再玩一次。

**規格**：
- 遊戲結束後詢問「再玩一次？」
- 接受 y/n 或 是/否
- 選擇「是」則重新開始
- 選擇「否」則顯示統計並結束

**技術實作**：

In [ ]:
def ask_play_again():
    """
    詢問是否再玩一次
    
    回傳：
        bool: True=再玩, False=結束
    """
    while True:
        choice = input("\n再玩一次？(y/n): ").strip().lower()
        
        if choice in ['y', 'yes', '是']:
            return True
        elif choice in ['n', 'no', '否']:
            return False
        else:
            print("請輸入 y 或 n！")

# 主程式框架
def main():
    """主程式"""
    total_games = 0
    total_wins = 0
    
    while True:
        # 遊戲邏輯...
        total_games += 1
        
        # if won:
        #     total_wins += 1
        
        # 詢問再玩一次
        if not ask_play_again():
            break
    
    # 顯示最終統計
    print(f"\n=== 遊戲統計 ===")
    print(f"總共玩了 {total_games} 局")
    print(f"勝利 {total_wins} 局")
    if total_games > 0:
        win_rate = (total_wins / total_games) * 100
        print(f"勝率：{win_rate:.2f}%")
    print("\n感謝遊玩！")

# 測試
# main()

**驗收標準**：
- [ ] 正確識別 y/n 輸入
- [ ] 支援多種輸入格式
- [ ] 統計資訊準確
- [ ] 流程順暢自然

---

### 需求 2.4：遊戲統計資訊

**描述**：記錄並顯示多局遊戲的統計資料。

**規格**：
- 總遊戲次數
- 勝利次數
- 勝率百分比
- 平均猜測次數（進階）
- 最少猜測次數（進階）

**技術實作**：

In [ ]:
class GameStats:
    """遊戲統計類別"""
    
    def __init__(self):
        self.total_games = 0
        self.total_wins = 0
        self.attempts_list = []  # 記錄每局猜測次數
    
    def add_game(self, won, attempts):
        """記錄一局遊戲"""
        self.total_games += 1
        if won:
            self.total_wins += 1
            self.attempts_list.append(attempts)
    
    def show_stats(self):
        """顯示統計資訊"""
        print("\n" + "="*50)
        print("📊 遊戲統計")
        print("="*50)
        print(f"總遊戲次數：{self.total_games} 局")
        print(f"勝利次數：{self.total_wins} 局")
        print(f"失敗次數：{self.total_games - self.total_wins} 局")
        
        if self.total_games > 0:
            win_rate = (self.total_wins / self.total_games) * 100
            print(f"勝率：{win_rate:.2f}%")
        
        if self.attempts_list:
            avg_attempts = sum(self.attempts_list) / len(self.attempts_list)
            print(f"平均猜測次數：{avg_attempts:.2f} 次")
            print(f"最佳紀錄：{min(self.attempts_list)} 次")
        
        print("="*50)

# 測試
stats = GameStats()
stats.add_game(won=True, attempts=3)
stats.add_game(won=True, attempts=5)
stats.add_game(won=False, attempts=7)
stats.add_game(won=True, attempts=2)
stats.show_stats()

**驗收標準**：
- [ ] 統計資料正確
- [ ] 百分比計算準確
- [ ] 處理除以零情況
- [ ] 顯示格式專業

---

## ✅ Part III: 完整測試案例

### 測試案例 1：正常遊戲流程

**情境**：玩家在 3 次內猜中答案

**測試步驟**：
1. 開始遊戲（答案假設為 63）
2. 輸入 50 → 顯示「太小了！」
3. 輸入 75 → 顯示「太大了！」
4. 輸入 63 → 顯示「答對了！你總共猜了 3 次」

**預期結果**：
- ✅ 猜測次數正確顯示
- ✅ 判斷邏輯正確
- ✅ 答對後遊戲結束
- ✅ 勝利訊息正確

---

### 測試案例 2：用完次數失敗

**情境**：玩家用完 7 次機會未猜中

**測試步驟**：
1. 開始遊戲（答案假設為 63）
2. 依序輸入：10, 20, 30, 40, 50, 60, 70
3. 第 7 次後遊戲結束

**預期結果**：
- ✅ 最多只能猜 7 次
- ✅ 顯示失敗訊息
- ✅ 公布正確答案
- ✅ 次數計算正確

---

### 測試案例 3：輸入錯誤處理

**情境**：測試各種錯誤輸入

**測試步驟**：
1. 輸入非數字 "abc" → 顯示錯誤，不計次數
2. 輸入 0 → 顯示範圍錯誤，不計次數
3. 輸入 101 → 顯示範圍錯誤，不計次數
4. 輸入 50 → 正常處理，計入次數

**預期結果**：
- ✅ 錯誤輸入不計入次數
- ✅ 錯誤訊息清楚
- ✅ 重新要求輸入
- ✅ 不會造成程式崩潰

---

### 測試案例 4：邊界值測試

**情境**：測試邊界數字

**測試步驟**：
1. 答案設為 1，輸入 1 → 答對
2. 答案設為 100，輸入 100 → 答對
3. 答案設為 1，輸入 2 → 太大
4. 答案設為 100，輸入 99 → 太小

**預期結果**：
- ✅ 邊界值可正常輸入
- ✅ 邊界值判斷正確
- ✅ 不會產生 off-by-one 錯誤

---

### 測試案例 5：難度選擇

**情境**：測試不同難度設定

**測試步驟**：
1. 選擇簡單難度 → 範圍 1-50，10 次
2. 選擇中等難度 → 範圍 1-100，7 次
3. 選擇困難難度 → 範圍 1-200，6 次

**預期結果**：
- ✅ 難度參數正確套用
- ✅ 隨機數在正確範圍
- ✅ 次數限制正確

---

### 測試案例 6：多局遊戲

**情境**：測試重新遊戲與統計

**測試步驟**：
1. 完成第 1 局（勝利，3 次）
2. 選擇再玩一次
3. 完成第 2 局（失敗，7 次）
4. 選擇結束

**預期結果**：
- ✅ 統計顯示 2 局
- ✅ 勝率 50%
- ✅ 平均猜測次數正確
- ✅ 最佳紀錄顯示 3 次

---

## 📋 需求檢核清單

### 基本需求
- [ ] 1.1 隨機數生成正確
- [ ] 1.2 輸入驗證完整
- [ ] 1.3 判斷邏輯正確
- [ ] 1.4 次數限制正確
- [ ] 1.5 結束訊息完整

### 進階需求
- [ ] 2.1 難度選擇功能
- [ ] 2.2 猜測歷史記錄
- [ ] 2.3 重新遊戲功能
- [ ] 2.4 統計資訊顯示

### 程式品質
- [ ] 程式碼符合 PEP 8
- [ ] 註解完整清楚
- [ ] 錯誤處理完善
- [ ] 使用者體驗友善

### 測試覆蓋
- [ ] 正常流程測試
- [ ] 邊界值測試
- [ ] 錯誤輸入測試
- [ ] 多局遊戲測試

---

**文件版本**：1.0  
**最後更新**：2025-10-05